In [45]:
import pandas as pd
from sqlalchemy import create_engine
from config import rds_connection_string

ModuleNotFoundError: No module named 'config'

In [46]:
rawdata_2008 = pd.read_csv('Data/data.csv')
rawdata_2018 =pd.read_csv('Data/zipcode_sales_nov2018.csv')

In [47]:
def clean_2018_data(rawdata_2018):
    cleandata_2018= (rawdata_2018.copy()
                         .rename(columns={"ZIP":'zip',
                                          "Sales":'Sales_2018',
                                          '2017 median':'Median_Price_2017',
                                          '2018 median':'Median_Price_2018'})
                         .drop(['Change'],axis=1))
    return cleandata_2018

In [48]:
def clean_2008_data(rawdata_2008):
    cleandata_2008 = rawdata_2008.copy()
    cleandata_2008 = cleandata_2008.drop(cleandata_2008.index[cleandata_2008['sq_ft']==0])
    
    # determine the price per sqft
    cleandata_2008['price per sqft'] = cleandata_2008['price']/cleandata_2008['sq_ft']
    
    return cleandata_2008

In [49]:
def aggregate_2008_data(cleandata_2008):
    aggregate_2008 = (round(cleandata_2008.copy()
         #group sales by zip code
        .groupby('zip')
         # find the medians of each category, and the total sales count                   
        .agg({'price':'median',
              'price per sqft':'median',
              'sq_ft':'median',
              'city':'count'})
         # rename column headers to reflect the new meanings                   
        .rename(columns={'city':"Sales_2008",
                         'price':"Median_Price_2008",
                         'price per sqft':'Median_Price_sqft_2008',
                         'sq_ft':"Median_sqft_2008"}),0)
                     )
    return aggregate_2008

In [50]:
def export_data(data_2008,data_2018,output_path):
    combined_stats = pd.merge(data_2008, data_2018, on="zip",how='inner')
    combined_stats.to_csv('Data/zipcode_medians_combined.csv')
    
    return combined_stats

In [53]:
def load_data(data):

    engine = create_engine(f'postgresql://{rds_connection_string}')
    combined_df.to_sql(name='zippy', con=engine, if_exists='append', index=False)

In [55]:
clean2018 = clean_2018_data(rawdata_2018)
clean2008 = clean_2008_data(rawdata_2008)
aggregate2008 = aggregate_2008_data(clean2008)
combined_df =export_data(aggregate2008,clean2018,output_path)
load_data(combined_df)

combined_df.head(3)

,zip,Median_Price_2008,Median_Price_sqft_2008,Median_sqft_2008,Sales_2008,Community,Sales_2018,Median_Price_2017,Median_Price_2018
0,95603,285000.0,207.0,1255.0,3,Auburn,29,407550,440000
1,95608,248567.0,182.0,1472.0,20,Carmichael,61,372250,399500
2,95610,219000.0,123.0,1410.0,7,Citrus Heights,29,325000,362000
